In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 87.0 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 66.1 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 62.3 MB/s 
     |████████████████████████████████| 127 kB 33.6 MB/s 
     |████████████████████████████████| 115 kB 81.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully unin

In [2]:
import tensorflow as tf
import numpy as np
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import json
import matplotlib.pyplot as plt
import random
import seaborn as sn
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import tokenizers
from datasets import load_dataset
from datasets import Dataset

In [3]:
# sarcasm_data_new = load_dataset("csv", data_files="/content/drive/MyDrive/datasets/train-balanced-sarcasm.csv")

# training_data = sarcasm_data_new["train"]["comment"]
# training_label = sarcasm_data_new["train"]["label"]

# dataset = sarcasm_data_new.train_test_split(test_size = 0.1)

In [20]:
sarcasm_data = pd.read_csv("/content/drive/MyDrive/thesis_datasets/train-balanced-sarcasm.csv")
sarcasm_data_test = pd.read_csv("/content/drive/MyDrive/thesis_datasets/test-balanced.csv")
print(sarcasm_data.shape)
# sarcasm_data.head() 
sarcasm_data_test

(1010826, 10)


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [9]:
sarcasm_data.drop(['author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'], axis=1, inplace=True)
# remove empty rows
sarcasm_data.dropna(inplace=True)
sarcasm_data.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [10]:
mispell_dict = {"ain't": "is not", "cannot": "can not", "aren't": "are not", "can't": "can not", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "this's": "this is", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is", "they'd": "they would",
                "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
                "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "wont": "will not", "won't've": "will not have", "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color',
                'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What',
                'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I',
                'theBest': 'the best', 'howdoes': 'how does', 'Etherium': 'Ethereum',
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what',
                'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

mispell_dict = {k.lower(): v.lower() for k, v in mispell_dict.items()}

In [11]:
# Let's make our preprocessing function.

def preprocessing_text(s):
    # making our string lowercase & removing extra spaces
    s = str(s).lower().strip()
    
    # remove contractions.
    s = " ".join([mispell_dict[word] if word in mispell_dict.keys() else word for word in s.split()])
    
    # removing \n
    s = re.sub('\n', '', s)
    
    # put spaces before & after punctuations to make words seprate. Like "king?" to "king", "?".
    s = re.sub(r"([?!,+=—&%\'\";:¿।।।|\(\){}\[\]//])", r" \1 ", s)
    
    # Remove more than 2 continues spaces with 1 space.
    s = re.sub('[ ]{2,}', ' ', s).strip()
    
    return s


# apply preprocessing_text function
sarcasm_data['comment'] = sarcasm_data['comment'].apply(preprocessing_text)
sarcasm_data.head()

,label,comment
0,0,nc and nh.
1,0,you do know west teams play against west teams...
2,0,"they were underdogs earlier today , but since ..."
3,0,"this meme is not funny none of the "" new york ..."
4,0,i could use one of those tools.


In [12]:
sarcasm_data = sarcasm_data.dropna()
sarcasm_data.count()

label      1010773
comment    1010773
dtype: int64

In [13]:
sarcasm_data = sarcasm_data.reset_index(drop=True)
data = sarcasm_data.drop(["label"], axis = 1)
label = sarcasm_data.drop(["comment"], axis = 1)
# dataset = Dataset.from_pandas(data)
# dataset

In [14]:
# df = data.assign(Index=range(len(data))).set_index('Index')
# df_lab = label.assign(Index=range(len(label))).set_index('Index')
# print(df.head())

In [15]:
train_text, val_text, train_labels, val_labels = train_test_split(data, label, test_size=0.1)

# below code doesnt help
# train_dataset = Dataset.from_pandas(train_text)
# val_dataset = Dataset.from_pandas(val_text)
# train_label = Dataset.from_pandas(train_labels)
# val_label = Dataset.from_pandas(val_labels)


In [43]:
# it is way to remove column from dataset dataframe/features
# train_dataset.features.pop("__index_level_0__")

In [44]:
# val_dataset.features.pop("__index_level_0__")

In [45]:
## this also doesnt help bcz still error
# tt = train_text.to_numpy()
# bb = val_text.to_numpy()

In [16]:
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# from transformers import RobertaTokenizerFast
# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# def tokenize_data(data):
#     return tokenizer(data['comment'], padding=True, truncation=True)

# # def tokenize_data(data):
# #     return tokenizer(data['comment'], padding="max_length", max_length=200, truncation=True)


# train_tokenized_data = train_dataset.map(tokenize_data, batched=True, batch_size=500)
# val_tokenized_data = val_dataset.map(tokenize_data, batched=True, batch_size=500)
# tokenized_data

train_dd = list(train_text["comment"])
val_dd = list(val_text["comment"])
train_label = list(train_labels["label"])
val_label = list(val_labels["label"])

# train_encodings = tokenizer(train_dataset, padding=True, truncation=True)
# val_encodings = tokenizer(val_dataset, padding=True, truncation=True)

train_encodings = tokenizer(train_dd, padding="max_length", truncation=True, max_length = 100)
val_encodings = tokenizer(val_dd, padding="max_length", truncation=True, max_length = 100)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_label
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_label
))

In [80]:
# train_dataset.shuffle(1000).batch(16)

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 100), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 100), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
c# dataset = Dataset.from_pandas(sarcasm_data)
# dataset = dataset.train_test_split(test_size = 0.2)

In [ ]:
# sarcasm_data = sarcasm_data.reset_index(drop=True)
# data = sarcasm_data.drop(["label"], axis = 1)
# label = sarcasm_data.drop(["comment"], axis = 1)
# dataset = Dataset.from_pandas(data)
# dataset

In [ ]:
# sarcasm_data = sarcasm_data.reset_index(drop=True)

# from sklearn.model_selection import train_test_split
# train_text, val_text, train_labels, val_labels = train_test_split(sarcasm_data['comment'], sarcasm_data['label'], test_size=0.2)

In [ ]:
# # df['column'] = df['column'].astype('str') 
# dd = list(train_text)
# bb = list(val_text)
# t_label = list(val_text)
# test_label = list(val_labels)

In [ ]:
# from transformers import RobertaTokenizer, TFRobertaModel
# import tensorflow as tf

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# def tokenize_data(data):
#     return tokenizer(data['comment'], padding=True, truncation=True)

# # def tokenize_data(data):
# #     return tokenizer(data['comment'], padding="max_length", max_length=200, truncation=True)


# tokenized_data = dataset.map(tokenize_data, batched=True, batch_size=500)
# tokenized_data

  0%|          | 0/2022 [00:00<?, ?ba/s]

Dataset({
    features: ['comment', 'input_ids', 'attention_mask'],
    num_rows: 1010773
})

In [ ]:
# from transformers import RobertaTokenizer, TFRobertaModel
# import tensorflow as tf

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# train_encodings = tokenizer(dd, padding=True, truncation=True)
# val_encodings = tokenizer(bb, padding=True, truncation=True)

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     t_label
# ))

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#     test_label
# ))

KeyboardInterrupt: ignored

In [ ]:
# dataset_preprocessed = tokenized_data.train_test_split(test_size = 0.2)
# dataset_preprocessed

DatasetDict({
    train: Dataset({
        features: ['comment', 'input_ids', 'attention_mask'],
        num_rows: 808618
    })
    test: Dataset({
        features: ['comment', 'input_ids', 'attention_mask'],
        num_rows: 202155
    })
})

In [73]:
import tensorflow.keras as keras 

encoder = TFRobertaModel.from_pretrained('roberta-base')

class TextClassificationModel(keras.Model):
  def __init__(self, encoder):
    super(TextClassificationModel, self).__init__()
    self.encoder = encoder
    self.encoder.trainable = False
    self.dropout1 = keras.layers.Dropout(0.1)
    self.dropout2 = keras.layers.Dropout(0.1)
    self.dense1 = keras.layers.Dense(64, activation = "relu")
    self.dense2 = keras.layers.Dense(2, activation = "softmax")

  def call(self,  input):
    x = self.encoder(input)
    x = x["last_hidden_state"][:, 0, :]
    x = self.dropout1(x)
    x = self.dense1(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    return x


# STEP 2

text_classification_model = TextClassificationModel(encoder)

text_classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                                  loss = "sparse_categorical_crossentropy",
                                  metrics=["accuracy"])

# checkpoint_path = "/content/training_2/cp.ckpt"
# # checkpoint_dir = os.path.dirname(checkpoint_path)

# checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
# earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# history = text_classification_model.fit(
#     train_dataset.shuffle(1000).batch(16), 
#     epochs=20,
#     validation_data= val_dataset.batch(16),
#     callbacks=[checkpoint, earlystopping, tensorboard_callback, cp_callback]
# )


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/20
50539/50539 [==============================] - ETA: 0s - loss: 0.6543 - accuracy: 0.6149
Epoch 1: val_accuracy improved from -inf to 0.66431, saving model to /content/training_2/cp.ckpt



Epoch 1: saving model to /content/training_2/cp.ckpt
50539/50539 [==============================] - 1898s 37ms/step - loss: 0.6543 - accuracy: 0.6149 - val_loss: 0.6172 - val_accuracy: 0.6643
Epoch 2/20
50538/50539 [============================>.] - ETA: 0s - loss: 0.6338 - accuracy: 0.6388
Epoch 2: val_accuracy improved from 0.66431 to 0.67164, saving model to /content/training_2/cp.ckpt



Epoch 2: saving model to /content/training_2/cp.ckpt
50539/50539 [==============================] - 1878s 37ms/step - loss: 0.6338 - accuracy: 0.6388 - val_loss: 0.6083 - val_accuracy: 0.6716
Epoch 3/20
50539/50539 [==============================] - ETA: 0s - loss: 0.6269 - accuracy: 0.6460
Epoch 3: val_accuracy improved from 0.67164 to 0.67852, saving model to /content/training_2/cp.ckpt



Epoch 3: saving model to /content/training_2/cp.ckpt
50539/50539 [==============================] - 1888s 37ms/step - loss: 0.6269 - accuracy: 0.6460 - val_loss: 0.6045 - val_accuracy: 0.6785
Epoch 4/20
50538/50539 [============================>.] - ETA: 0s - loss: 0.6228 - accuracy: 0.6502
Epoch 4: val_accuracy improved from 0.67852 to 0.68221, saving model to /content/training_2/cp.ckpt



Epoch 4: saving model to /content/training_2/cp.ckpt
50539/50539 [==============================] - 1885s 37ms/step - loss: 0.6228 - accuracy: 0.6502 - val_loss: 0.5970 - val_accuracy: 0.6822
Epoch 5/20
50538/50539 [============================>.] - ETA: 0s - loss: 0.6203 - accuracy: 0.6521
Epoch 5: val_accuracy improved from 0.68221 to 0.68309, saving model to /content/training_2/cp.ckpt



Epoch 5: saving model to /content/training_2/cp.ckpt
50539/50539 [==============================] - 1893s 37ms/step - loss: 0.6203 - accuracy: 0.6521 - val_loss: 0.5958 - val_accuracy: 0.6831
Epoch 6/20
  345/50539 [..............................] - ETA: 25:07 - loss: 0.6241 - accuracy: 0.6520

KeyboardInterrupt: ignored

In [143]:
# Save the weights
text_classification_model.save_weights('/content/drive/MyDrive/Colab Notebooks/Thesis and paper_work/trained_weights/ro1')

In [131]:
# Evaluate

# Evaluate the model
# loss, acc = text_classification_model.evaluate(val_dataset.batch(32), steps = 200, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# Evaluate the model
loss, acc = text_classification_model.evaluate(val_dataset.batch(32), verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# result = text_classification_model.predict(val_dataset.batch(32), steps = 20)
# print("Validation accuracy: ", sklearn.metrics.accuracy_score(y_val, y_preds))

6318/6318 - 222s - loss: 0.5961 - accuracy: 0.6841 - 222s/epoch - 35ms/step
Restored model, accuracy: 68.41%


In [99]:
# ff = text_classification_model(next(iter(val_dataset.batch(4)))).numpy()
# ff.argmax(axis=-1)

array([0, 1, 1, 1])

In [119]:
ff = text_classification_model(next(iter(val_dataset.batch(16)))).numpy()
ff.argmax(axis=-1)

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [154]:
# Restore the weights
# model = text_classification_model.load_weights('/content/drive/MyDrive/Colab Notebooks/Thesis and paper_work/trained_weights/ro1')
# model.load_weights
# load the complete model
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis and paper_work/saved_models/roberta5_epochs')

# Evaluate the model
loss, acc = new_model.evaluate(val_dataset.batch(32), verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

TypeError: ignored

In [ ]:
# Save the entire model
text_classification_model.save('/content/drive/MyDrive/Colab Notebooks/Thesis and paper_work/saved_models/roberta5_epochs')

In [99]:
# to convert from tensor to numpy use tensor.numpy()
# to get the prediction with max value is  ff.argmax(axis=-1)

In [ ]:
# from transformers import RobertaTokenizer, TFRobertaModel
# import tensorflow as tf

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = TFRobertaModel.from_pretrained('roberta-base')

# # last_hidden_states = outputs.last_hidden_state